In [22]:
import pandas as pd
import numpy as np
import os
import datetime
import time

In [23]:
import git
import sys
repo = git.Repo("./", search_parent_directories=True)
homedir = repo.working_dir

In [24]:
#Neighbor Data
neighborcounties = pd.read_csv(f"{homedir}/models/processing/USA/County_Based/neighborcounties.csv", index_col = 0)
#Google Data
Google = pd.read_csv(f"{homedir}/models/processing/USA/County_Based/google_new.csv", index_col = 0)
#Fixed Data
Age_Race = pd.read_csv(f"{homedir}/models/processing/USA/County_Based/Age_Race.csv", index_col = 0)
Pop_60 = pd.read_csv(f"{homedir}/models/processing/USA/County_Based/Pop_60.csv")
Density = pd.read_csv(f"{homedir}/models/processing/USA/County_Based/Density.csv")

In [25]:
def fillframe_date(fips, date, value, colname, neighborcounties, min_neighbors=2):
    #Is able to fill a dataframe that has a date column by pivoting by date
    #And filling in NaNs by factor & date for each county, using fillcol method
    #Making Dataframe to sort values by dates, pivoting
    tic = time.time()
    frame = pd.DataFrame(data = [fips,date,value])
    frame = frame.T
    frame.columns = ['FIPS', 'Date', colname]
    frame = frame.pivot(index = 'FIPS', columns='Date', values=[colname]).reset_index()
    
    #fixing up the frame to haev proper columns
    frame.to_csv('frame.csv')
    frame = pd.read_csv('frame.csv', encoding='latin1')
    frame = frame.rename(columns=frame.iloc[0]).drop(frame.index[0])
    frame.columns = frame.columns.fillna('FIPS')
    frame = frame.drop(columns = 'Date')
    toc = time.time()
    print(toc - tic)
    #Iterating through each date to fill NaNs
    datelist = frame.columns    
    datelist = datelist[1:]
    for d in datelist:
        print(d)
        tic = time.time()
        frame[d] = fillcol(frame['FIPS'], frame[d], neighborcounties, min_neighbors)
        toc = time.time()
        print(toc - tic)
    #Unpivoting the frame so it can be returned
    frame = frame.melt(id_vars=['FIPS']).sort_values(['FIPS','variable']).reset_index(drop=True)
    frame.columns = ['FIPS', 'Date', colname]
    
    return frame

In [26]:
def fillcol(fips, value,neighborcounties, min_neighbors=2):
    #Takes in a column of fips codes, and any type of datafield with some NaNs,
    #Computes distance-weighted average of the value across all neighbors of NaN counties
    tic1 = time.time()
    #Loading in the fips and value into proper dataframes
    #This is the df with only nan values
    df = pd.DataFrame(data = [fips,value]).T
    df.columns = ['FIPS', 'Values']
    df.Values = df.Values.astype(float)
    df = df.set_index('FIPS')
    
    #creating new column to set to the current dataframe values
    newcol = []
    for ind in df.index:
        #for any entries with NaNs
        if np.isnan(df['Values'][ind]):
            #list of neighbors for NaN county
            neighbors = list(neighborcounties[neighborcounties['orgfips'] == ind]['adjfips'])
            nonzero = 0
            weightedval = 0
            totalinvdist = 0
            totaldist = 0
            vals = 0
            #iterates though neighbors of NaN county with non-NaN entires
            for n in neighbors:
                if n in df.index:
                    if ~np.isnan(df['Values'][n]):
                        #Getting weighted values, using 1/dist as a scalar to show closer distance counts more
                        nonzero += 1
                        dist = list(neighborcounties.query('orgfips == ' + str(ind) + ' and adjfips == ' + str(n))['Pop_10'])[0]
                        totalinvdist += (1/dist)**1
                        weightedval += ((1/dist)**1)*df['Values'][n]
            #If there are at least 2 neighbors (this can be adjusted)
            if nonzero >= min_neighbors:
                newcol.append(weightedval/(totalinvdist))
            else:
                newcol.append(np.nan)
        else:
            newcol.append(df['Values'][ind])
    toc1 = time.time()
    print(toc1 - tic1)
    return newcol

In [27]:
def fillgoogle(colname, google_neighbor):
    #Method to fill up the google mobility data
    #Uses colname to designate which column to fill
    #rows that have NaN values at the value of interest
    numnans = len(google_neighbor[np.isnan(google_neighbor[colname])])
    while numnans > 0:
        print('1: ' + str(numnans))
        tempnum = numnans
        #Creating the filled column from method
        temp = fillframe_date(google_neighbor['FIPS'],google_neighbor['Date'],google_neighbor[colname], colname, neighborcounties)
        google_neighbor[colname] = temp[colname]
        numnans = len(google_neighbor[np.isnan(google_neighbor[colname])])
        print('2: ' + str(numnans))
        #Checking if the number of nans changes
        if tempnum == numnans:
            print(numnans)
            print('3: ' + str(numnans))
            #if number doesnt change, try again with only 1 neighbor, otherwise quit
            temp = fillframe_date(google_neighbor['FIPS'],google_neighbor['Date'],google_neighbor[colname], colname, neighborcounties, 1)
            google_neighbor[colname] = temp[colname]
            numnans = len(google_neighbor[np.isnan(google_neighbor[colname])])
            print('4: ' + str(numnans))
            if tempnum <= numnans:
                numnans = 0
        elif tempnum < numnans:
            numnans = 0
        
    return google_neighbor

In [28]:
def fillfixed(colname, data, code, neighborcounties):
    #Method to fill up the google mobility data
    #Uses colname to designate which column to fill
    numnans = len(data[np.isnan(data[colname])])
    while numnans > 0:
        print(numnans)
        tempnum = numnans
        #Creating the filled column from method
        newcol = fillcol(data[code], data[colname], neighborcounties)
        data[colname] = newcol
        numnans = len(data[np.isnan(data[colname])])
        #Checking if the number of nans changes
        if tempnum == numnans:
            #if number doesnt change, try again with only 1 neighbor, otherwise quit
            newcol = fillcol(data[code], data[colname], neighborcounties)
            data[colname] = newcol
            numnans = len(data[np.isnan(data[colname])])
            if tempnum == numnans:
                numnans = 0     
    return data    

In [29]:
def popfill(total,estimate):
    #Fills in the population of each county, merging density with Pop_60
    if np.isnan(total):
        return estimate
    else:
        return total

In [30]:
#List of all the counties with neighbor data
county_list = pd.DataFrame(neighborcounties.orgfips.unique(),columns = ['fips']).set_index('fips')
#Calculating the Total population for each county
Total_Pop = county_list.join(Pop_60.set_index('FIPS'), how='left')
#Adding in extra approximated rows for NaN values, using Density
Density['Pop'] = Density['2010 Density per square mile of land area - Population'] * Density['Area in square miles - Land area']
Density = Density[['FIPS', 'Pop']]
Total_Pop = Total_Pop.join(Density.set_index('FIPS'), how='left')
Total_Pop['Population'] = Total_Pop.apply(lambda x: popfill(x['total_pop'],x['Pop']), axis=1)
Total_Pop = Total_Pop['Population']
Total_Pop.to_csv('Total_Pop')

C:\Users\jethi\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  # This is added back by InteractiveShellApp.init_path()


In [31]:
#Adding extra rows to demographic info to include all counties
Age_Race = county_list.join(Age_Race, how='left')
#Setting the total population for each county
Age_Race['Total population'] = list(Total_Pop)
#Dividing the entires by population
Age_Race = Age_Race.astype(str).astype(float)
Age_Race = Age_Race.divide(Age_Race['Total population'], axis=0).reset_index()

#Going through each column of Age_Race to fill NaNs
cols = list(Age_Race.columns)[2:]
for col in cols:
    Age_Race = fillfixed(col, Age_Race, 'fips', neighborcounties)
Age_Race = Age_Race.dropna()

#I Ran this halfway and then stopped

2383


KeyboardInterrupt: 

In [135]:
Age_Race.to_csv('Age_Race_Filled.csv')

In [129]:
cols = list(Google.columns)[2:]

In [175]:
Google = fillgoogle(cols[0], Google)
Google

1: 26038
14.493613243103027
2020-02-15
7.188775062561035
7.191766738891602
2020-02-16
8.09603762626648
8.10304069519043
2020-02-17
7.216412305831909
7.219425916671753
2020-02-18
7.179968595504761
7.182960271835327
2020-02-19
6.837110757827759
6.839104890823364
2020-02-20
6.942372560501099
6.944366455078125
2020-02-21
6.140990972518921
6.145971775054932
2020-02-22
6.580541133880615
6.584521532058716
2020-02-23
7.513809442520142
7.515824556350708
2020-02-24
7.320533037185669
7.3235204219818115
2020-02-25
7.80838680267334
7.8103814125061035
2020-02-26
8.03112244606018
8.034123182296753
2020-02-27
7.97749137878418
7.980483770370483
2020-02-28
7.342360973358154
7.34535026550293
2020-02-29
7.465514659881592
7.469501495361328
2020-03-01
8.413213968276978
8.4172043800354
2020-03-02
6.761101961135864
6.76409387588501
2020-03-03
7.5945494174957275
7.599606513977051
2020-03-04
7.239335298538208
7.240345001220703
2020-03-05
7.343762397766113
7.345757246017456
2020-03-06
6.7080183029174805
6.709015

6.620858907699585
6.621856689453125
2: 785
1: 785
10.791979551315308
2020-02-15
3.5334208011627197
3.5374045372009277
2020-02-16
4.559070825576782
4.562054872512817
2020-02-17
1.3843011856079102
1.3882911205291748
2020-02-18
1.757012128829956
1.7590456008911133
2020-02-19
1.5871186256408691
1.5901374816894531
2020-02-20
1.478759527206421
1.4808154106140137
2020-02-21
1.918588399887085
1.9215819835662842
2020-02-22
3.5139615535736084
3.5169808864593506
2020-02-23
4.584223985671997
4.586258172988892
2020-02-24
1.8188073635101318
1.8217976093292236
2020-02-25
1.8004984855651855
1.8024966716766357
2020-02-26
1.701249599456787
1.703212022781372
2020-02-27
1.717857837677002
1.719848871231079
2020-02-28
1.9109365940093994
1.9129321575164795
2020-02-29
3.6796419620513916
3.6816415786743164
2020-03-01
4.735564947128296
4.737592458724976
2020-03-02
1.8265938758850098
1.8285894393920898
2020-03-03
1.7088277339935303
1.7108330726623535
2020-03-04
1.6087019443511963
1.609701156616211
2020-03-05
1.6

0.8477675914764404
0.8487660884857178
2020-04-10
1.2147247791290283
1.2157223224639893
2020-04-11
6.070946216583252
6.072951555252075
2: 752
752
3: 752
10.189733982086182
2020-02-15
3.515994071960449
3.518975019454956
2020-02-16
4.489920139312744
4.492910623550415
2020-02-17
1.3897995948791504
1.3927924633026123
2020-02-18
1.723914384841919
1.7259149551391602
2020-02-19
1.5552184581756592
1.5572137832641602
2020-02-20
1.4617035388946533
1.4646580219268799
2020-02-21
1.89194655418396
1.8949015140533447
2020-02-22
3.490648031234741
3.4936776161193848
2020-02-23
4.493041515350342
4.495044231414795
2020-02-24
1.770073413848877
1.7730984687805176
2020-02-25
1.7697293758392334
1.7727620601654053
2020-02-26
1.6656277179718018
1.66861891746521
2020-02-27
1.6481194496154785
1.6501109600067139
2020-02-28
1.8572430610656738
1.8592474460601807
2020-02-29
3.631075382232666
3.633070468902588
2020-03-01
4.751669406890869
4.753698110580444
2020-03-02
1.8560147285461426
1.8580517768859863
2020-03-03
1.

,FIPS,Date,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,residential_percent_change_from_baseline,retail_and_recreation_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline
0,1001,2020-02-15,7.000000,NaN,NaN,5.0,NaN,-4.0
1,1001,2020-02-16,1.000000,-23.0,NaN,0.0,NaN,-4.0
2,1001,2020-02-17,0.000000,NaN,5.0,8.0,NaN,-27.0
3,1001,2020-02-18,0.000000,NaN,0.0,-2.0,NaN,2.0
4,1001,2020-02-19,0.000000,NaN,0.0,-2.0,NaN,2.0
5,1001,2020-02-20,-3.000000,NaN,2.0,-8.0,NaN,1.0
6,1001,2020-02-21,-2.000000,NaN,0.0,-3.0,NaN,3.0
7,1001,2020-02-22,1.000000,NaN,NaN,1.0,NaN,8.0
8,1001,2020-02-23,3.000000,NaN,NaN,0.0,NaN,5.0
9,1001,2020-02-24,-5.000000,NaN,1.0,-2.0,NaN,2.0


In [176]:
Google = fillgoogle(cols[1], Google)
Google

1: 114449
10.007899284362793
2020-02-15
7.679256439208984
7.681210994720459
2020-02-16
7.249401092529297
7.251394748687744
2020-02-17
7.145565986633301
7.14752459526062
2020-02-18
7.237182378768921
7.239202260971069
2020-02-19
7.044832468032837
7.046792268753052
2020-02-20
7.380990505218506
7.382957220077515
2020-02-21
7.4437925815582275
7.4457926750183105
2020-02-22
7.50799036026001
7.510983228683472
2020-02-23
7.43186092376709
7.433829307556152
2020-02-24
7.402682065963745
7.403686761856079
2020-02-25
7.028169631958008
7.030184745788574
2020-02-26
7.071600437164307
7.073594331741333
2020-02-27
7.009655237197876
7.01164984703064
2020-02-28
7.346397638320923
7.348429203033447
2020-02-29
7.502607583999634
7.50359582901001
2020-03-01
6.974663972854614
6.976685523986816
2020-03-02
7.067033052444458
7.069027662277222
2020-03-03
6.745154142379761
6.746184349060059
2020-03-04
6.893751382827759
6.894749641418457
2020-03-05
6.9476869106292725
6.948720455169678
2020-03-06
7.160167694091797
7.16

10.373912334442139
2020-02-15
6.105877637863159
6.109880208969116
2020-02-16
6.755672454833984
6.758700370788574
2020-02-17
4.544470310211182
4.5484983921051025
2020-02-18
4.642721176147461
4.645714044570923
2020-02-19
4.78376317024231
4.786760091781616
2020-02-20
4.71322226524353
4.717211008071899
2020-02-21
4.80710244178772
4.810058355331421
2020-02-22
6.05929970741272
6.062356948852539
2020-02-23
6.736055135726929
6.740083456039429
2020-02-24
4.863905906677246
4.866903305053711
2020-02-25
4.728876829147339
4.731869459152222
2020-02-26
4.906018972396851
4.908973217010498
2020-02-27
4.874308347702026
4.87733793258667
2020-02-28
4.83318829536438
4.836183786392212
2020-02-29
6.159009218215942
6.162027359008789
2020-03-01
6.885205268859863
6.887229681015015
2020-03-02
4.951661825180054
4.954657316207886
2020-03-03
4.609362840652466
4.611845016479492
2020-03-04
4.647736072540283
4.649730443954468
2020-03-05
4.701215028762817
4.703208923339844
2020-03-06
4.923199653625488
4.927186012268066

6.103679180145264
6.105722665786743
2: 870
1: 870
10.527504920959473
2020-02-15
3.5870213508605957
3.5900144577026367
2020-02-16
4.566812515258789
4.570784330368042
2020-02-17
1.449209213256836
1.453226089477539
2020-02-18
1.7500779628753662
1.7540602684020996
2020-02-19
1.58270263671875
1.5867125988006592
2020-02-20
1.4767355918884277
1.4807238578796387
2020-02-21
1.8986186981201172
1.9016163349151611
2020-02-22
3.529815435409546
3.5337774753570557
2020-02-23
4.577276706695557
4.5803046226501465
2020-02-24
1.8116652965545654
1.8146615028381348
2020-02-25
1.8179948329925537
1.8209559917449951
2020-02-26
1.7259485721588135
1.7299036979675293
2020-02-27
1.763155460357666
1.7661871910095215
2020-02-28
1.900031566619873
1.903014898300171
2020-02-29
3.6791727542877197
3.6821653842926025
2020-03-01
4.7435243129730225
4.746516942977905
2020-03-02
1.8735558986663818
1.8765482902526855
2020-03-03
1.6935679912567139
1.6965603828430176
2020-03-04
1.6093225479125977
1.6123156547546387
2020-03-05
1

0.9589743614196777
0.959972620010376
2020-04-09
0.8612048625946045
0.8612048625946045
2020-04-10
1.2255196571350098
1.226527214050293
2020-04-11
6.146727561950684
6.147724628448486
2: 764
1: 764
10.712913990020752
2020-02-15
3.5685863494873047
3.572572946548462
2020-02-16
4.504890203475952
4.507883071899414
2020-02-17
1.4071390628814697
1.4101316928863525
2020-02-18
1.7286698818206787
1.7326600551605225
2020-02-19
1.5625629425048828
1.5655505657196045
2020-02-20
1.4535479545593262
1.4565403461456299
2020-02-21
1.877497673034668
1.8804969787597656
2020-02-22
3.500561237335205
3.503554105758667
2020-02-23
4.484299898147583
4.487292289733887
2020-02-24
1.7813303470611572
1.7833244800567627
2020-02-25
1.7743034362792969
1.7772953510284424
2020-02-26
1.6948347091674805
1.698824167251587
2020-02-27
1.6610901355743408
1.664086103439331
2020-02-28
1.8595664501190186
1.862560510635376
2020-02-29
3.6598377227783203
3.661834239959717
2020-03-01
4.789461851119995
4.792456865310669
2020-03-02
1.841

4.180733919143677
4.18177604675293
2020-04-06
1.2128541469573975
1.2148783206939697
2020-04-07
0.9955403804779053
0.9965376853942871
2020-04-08
0.9838693141937256
0.984891414642334
2020-04-09
0.8432285785675049
0.8442623615264893
2020-04-10
1.2299425601959229
1.2309401035308838
2020-04-11
6.071991682052612
6.073986053466797
2: 754
1: 754
10.282284021377563
2020-02-15
3.83607816696167
3.8410654067993164
2020-02-16
4.495529413223267
4.4985222816467285
2020-02-17
1.3839170932769775
1.3879475593566895
2020-02-18
1.7149057388305664
1.7179358005523682
2020-02-19
1.570202112197876
1.5742287635803223
2020-02-20
1.4986379146575928
1.5026543140411377
2020-02-21
1.9096174240112305
1.9125947952270508
2020-02-22
3.5259130001068115
3.5289058685302734
2020-02-23
4.542529106140137
4.545521974563599
2020-02-24
1.8017077445983887
1.8056957721710205
2020-02-25
1.7729902267456055
1.7759909629821777
2020-02-26
1.6739916801452637
1.6770226955413818
2020-02-27
1.6671671867370605
1.6711573600769043
2020-02-28

1.1255381107330322
1.127497911453247
2020-04-04
3.2711167335510254
3.273087978363037
2020-04-05
4.141340494155884
4.142327308654785
2020-04-06
1.2223279476165771
1.22432279586792
2020-04-07
0.9811325073242188
0.9821140766143799
2020-04-08
0.9526424407958984
0.9546501636505127
2020-04-09
0.8431589603424072
0.8441562652587891
2020-04-10
1.2262301445007324
1.2262301445007324
2020-04-11
6.015988826751709
6.0179829597473145
2: 746
1: 746
10.24719500541687
2020-02-15
3.5695717334747314
3.573561191558838
2020-02-16
4.492054224014282
4.495046615600586
2020-02-17
1.3771162033081055
1.3801085948944092
2020-02-18
1.725790023803711
1.7287817001342773
2020-02-19
1.570061206817627
1.5730535984039307
2020-02-20
1.4571216106414795
1.461111068725586
2020-02-21
1.8788504600524902
1.8818798065185547
2020-02-22
3.4786343574523926
3.481614828109741
2020-02-23
4.584307670593262
4.587335586547852
2020-02-24
1.789001703262329
1.7920210361480713
2020-02-25
1.7635829448699951
1.7665433883666992
2020-02-26
1.689

0.9314620494842529
0.933457612991333
2020-04-01
0.9739029407501221
0.975898027420044
2020-04-02
0.8432543277740479
0.8442513942718506
2020-04-03
1.1185274124145508
1.1195249557495117
2020-04-04
3.307612657546997
3.308610200881958
2020-04-05
4.137996673583984
4.138994216918945
2020-04-06
1.2165751457214355
1.2175726890563965
2020-04-07
0.9784398078918457
0.9794373512268066
2020-04-08
0.9444794654846191
0.945476770401001
2020-04-09
0.8325603008270264
0.8335540294647217
2020-04-10
1.2032606601715088
1.2042768001556396
2020-04-11
6.0876240730285645
6.088621139526367
2: 742
1: 742
10.363208532333374
2020-02-15
3.5407252311706543
3.544715642929077
2020-02-16
4.37085747718811
4.374846458435059
2020-02-17
1.3493080139160156
1.3523008823394775
2020-02-18
1.6788158416748047
1.6838021278381348
2020-02-19
1.5387182235717773
1.542708158493042
2020-02-20
1.4378488063812256
1.4408411979675293
2020-02-21
1.9173991680145264
1.921388864517212
2020-02-22
3.444929838180542
3.4479219913482666
2020-02-23
4.

3.324882984161377
3.326862335205078
2020-03-29
4.176156282424927
4.177117347717285
2020-03-30
1.0082595348358154
1.0102901458740234
2020-03-31
0.925501823425293
0.9274580478668213
2020-04-01
0.9632124900817871
0.9641735553741455
2020-04-02
0.8418223857879639
0.8445916175842285
2020-04-03
1.1405534744262695
1.1425316333770752
2020-04-04
3.291682004928589
3.292680263519287
2020-04-05
4.158758163452148
4.16071891784668
2020-04-06
1.2158384323120117
1.2168092727661133
2020-04-07
0.9709961414337158
0.9729890823364258
2020-04-08
0.953941822052002
0.9559102058410645
2020-04-09
0.8444023132324219
0.8453688621520996
2020-04-10
1.208115816116333
1.208115816116333
2020-04-11
6.063391923904419
6.065327882766724
4: 277
1: 277
10.198838710784912
2020-02-15
3.431159734725952
3.4351534843444824
2020-02-16
4.36477255821228
4.36873459815979
2020-02-17
1.2566981315612793
1.2606556415557861
2020-02-18
1.5034844875335693
1.5074727535247803
2020-02-19
1.4776759147644043
1.4806711673736572
2020-02-20
1.33528

,FIPS,Date,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,residential_percent_change_from_baseline,retail_and_recreation_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline
0,1001,2020-02-15,7.000000,5.522819,NaN,5.0,NaN,-4.0
1,1001,2020-02-16,1.000000,-23.000000,NaN,0.0,NaN,-4.0
2,1001,2020-02-17,0.000000,3.275069,5.0,8.0,NaN,-27.0
3,1001,2020-02-18,0.000000,-4.000000,0.0,-2.0,NaN,2.0
4,1001,2020-02-19,0.000000,5.016194,0.0,-2.0,NaN,2.0
5,1001,2020-02-20,-3.000000,-23.637534,2.0,-8.0,NaN,1.0
6,1001,2020-02-21,-2.000000,22.702300,0.0,-3.0,NaN,3.0
7,1001,2020-02-22,1.000000,32.506121,NaN,1.0,NaN,8.0
8,1001,2020-02-23,3.000000,3.165041,NaN,0.0,NaN,5.0
9,1001,2020-02-24,-5.000000,-26.055014,1.0,-2.0,NaN,2.0


In [177]:
Google = fillgoogle(cols[2], Google)
Google

1: 83002
10.054382085800171
2020-02-15
8.372480869293213
8.375499963760376
2020-02-16
9.36778473854065
9.370784759521484
2020-02-17
9.697264671325684
9.699219703674316
2020-02-18
9.550184726715088
9.552179098129272
2020-02-19
9.463996410369873
9.466028451919556
2020-02-20
8.921232461929321
8.923267602920532
2020-02-21
9.095388889312744
9.098404169082642
2020-02-22
8.354936361312866
8.355934858322144
2020-02-23
8.187339305877686
8.18937063217163
2020-02-24
8.987138271331787
8.98913049697876
2020-02-25
8.933970928192139
8.935950756072998
2020-02-26
8.962136268615723
8.964128255844116
2020-02-27
8.97105884552002
8.973053455352783
2020-02-28
9.026834487915039
9.028862953186035
2020-02-29
8.429713010787964
8.431707859039307
2020-03-01
8.211542844772339
8.213573455810547
2020-03-02
8.844566345214844
8.846560716629028
2020-03-03
8.895463466644287
8.897479057312012
2020-03-04
8.863062620162964
8.864051342010498
2020-03-05
8.862838745117188
8.864859104156494
2020-03-06
8.885871887207031
8.88786

4.651608943939209
4.654595375061035
2020-02-17
1.5720601081848145
1.57505202293396
2020-02-18
1.8818962574005127
1.88588547706604
2020-02-19
1.7707915306091309
1.7737562656402588
2020-02-20
1.679544448852539
1.6825406551361084
2020-02-21
2.1175057888031006
2.1195008754730225
2020-02-22
3.667323350906372
3.6703147888183594
2020-02-23
4.6843202114105225
4.688299655914307
2020-02-24
2.0251481533050537
2.0281667709350586
2020-02-25
1.9698469638824463
1.9718425273895264
2020-02-26
1.9198963642120361
1.9239203929901123
2020-02-27
2.0139262676239014
2.0179378986358643
2020-02-28
2.0742084980010986
2.076204299926758
2020-02-29
3.853278875350952
3.856271982192993
2020-03-01
4.92046594619751
4.924489974975586
2020-03-02
2.0691068172454834
2.071138381958008
2020-03-03
1.8855228424072266
1.88755464553833
2020-03-04
1.8519163131713867
1.8539049625396729
2020-03-05
1.9251112937927246
1.9281060695648193
2020-03-06
2.088395595550537
2.0903964042663574
2020-03-07
3.872453451156616
3.87448787689209
2020

6.065833806991577
6.0668699741363525
2: 766
1: 766
10.197847604751587
2020-02-15
3.516573667526245
3.5196034908294678
2020-02-16
4.5177531242370605
4.521765232086182
2020-02-17
1.3861591815948486
1.3891518115997314
2020-02-18
1.7349064350128174
1.738905429840088
2020-02-19
1.5762526988983154
1.5792722702026367
2020-02-20
1.4658818244934082
1.4678771495819092
2020-02-21
1.8782501220703125
1.8812358379364014
2020-02-22
3.5029242038726807
3.50691294670105
2020-02-23
4.55155348777771
4.554545640945435
2020-02-24
1.802854299545288
1.8058831691741943
2020-02-25
1.799159049987793
1.802152156829834
2020-02-26
1.6711292266845703
1.6741502285003662
2020-02-27
1.656766653060913
1.6587979793548584
2020-02-28
1.8741919994354248
1.8782188892364502
2020-02-29
3.681522846221924
3.6845288276672363
2020-03-01
4.738645792007446
4.741665601730347
2020-03-02
1.8366987705230713
1.8387341499328613
2020-03-03
1.6827874183654785
1.6847748756408691
2020-03-04
1.6141109466552734
1.6161062717437744
2020-03-05
1.6

0.9472732543945312
0.948270320892334
2020-04-09
0.8453307151794434
0.846325159072876
2020-04-10
1.2343957424163818
1.2343957424163818
2020-04-11
6.070579767227173
6.072574853897095
4: 284
1: 284
10.201252698898315
2020-02-15
3.506117820739746
3.509110927581787
2020-02-16
4.395162105560303
4.398156404495239
2020-02-17
1.2551703453063965
1.258162498474121
2020-02-18
1.5028436183929443
1.504838466644287
2020-02-19
1.4559736251831055
1.4589660167694092
2020-02-20
1.3308866024017334
1.33388090133667
2020-02-21
1.762826681137085
1.7658193111419678
2020-02-22
3.471820116043091
3.475809097290039
2020-02-23
4.552547216415405
4.554541826248169
2020-02-24
1.7188348770141602
1.7218284606933594
2020-02-25
1.591705322265625
1.5947322845458984
2020-02-26
1.5276377201080322
1.531618356704712
2020-02-27
1.5621285438537598
1.5661182403564453
2020-02-28
1.751378059387207
1.7543706893920898
2020-02-29
3.5369558334350586
3.5399484634399414
2020-03-01
4.556303262710571
4.5592968463897705
2020-03-02
1.715481

,FIPS,Date,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,residential_percent_change_from_baseline,retail_and_recreation_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline
0,1001,2020-02-15,7.000000,5.522819,-1.527507,5.0,NaN,-4.0
1,1001,2020-02-16,1.000000,-23.000000,1.000000,0.0,NaN,-4.0
2,1001,2020-02-17,0.000000,3.275069,5.000000,8.0,NaN,-27.0
3,1001,2020-02-18,0.000000,-4.000000,0.000000,-2.0,NaN,2.0
4,1001,2020-02-19,0.000000,5.016194,0.000000,-2.0,NaN,2.0
5,1001,2020-02-20,-3.000000,-23.637534,2.000000,-8.0,NaN,1.0
6,1001,2020-02-21,-2.000000,22.702300,0.000000,-3.0,NaN,3.0
7,1001,2020-02-22,1.000000,32.506121,-2.000000,1.0,NaN,8.0
8,1001,2020-02-23,3.000000,3.165041,-0.472493,0.0,NaN,5.0
9,1001,2020-02-24,-5.000000,-26.055014,1.000000,-2.0,NaN,2.0


In [178]:
Google = fillgoogle(cols[3], Google)
Google

1: 21667
10.185859203338623
2020-02-15
5.068335294723511
5.070329666137695
2020-02-16
6.407623291015625
6.40961766242981
2020-02-17
5.61188006401062
5.614872694015503
2020-02-18
5.684518814086914
5.686513423919678
2020-02-19
5.6123716831207275
5.615358591079712
2020-02-20
5.083824157714844
5.086877107620239
2020-02-21
4.375298738479614
4.378230094909668
2020-02-22
5.175773859024048
5.177751779556274
2020-02-23
6.704657554626465
6.7076499462127686
2020-02-24
5.605278253555298
5.607304811477661
2020-02-25
5.2409586906433105
5.242990255355835
2020-02-26
5.3619842529296875
5.362975835800171
2020-02-27
5.091940879821777
5.093972206115723
2020-02-28
4.509227752685547
4.511226654052734
2020-02-29
5.235008716583252
5.237029552459717
2020-03-01
6.727356910705566
6.7283806800842285
2020-03-02
5.708966493606567
5.710997104644775
2020-03-03
5.561098575592041
5.564034461975098
2020-03-04
5.340058326721191
5.34209132194519
2020-03-05
5.1089677810668945
5.1109619140625
2020-03-06
4.453944206237793
4.

6.106616020202637
6.107605695724487
2: 746
1: 746
10.165035963058472
2020-02-15
3.5171468257904053
3.5201761722564697
2020-02-16
4.524783611297607
4.527803421020508
2020-02-17
1.3861544132232666
1.3891868591308594
2020-02-18
1.705721139907837
1.7086727619171143
2020-02-19
1.5873053073883057
1.5893001556396484
2020-02-20
1.4832978248596191
1.4852964878082275
2020-02-21
1.896535873413086
1.8995544910430908
2020-02-22
3.4987943172454834
3.5008223056793213
2020-02-23
4.52490496635437
4.527905464172363
2020-02-24
1.7534995079040527
1.7564918994903564
2020-02-25
1.7922961711883545
1.7942910194396973
2020-02-26
1.6882939338684082
1.6922829151153564
2020-02-27
1.685481071472168
1.6864778995513916
2020-02-28
1.8698928356170654
1.8728852272033691
2020-02-29
3.6973342895507812
3.699329376220703
2020-03-01
4.784065008163452
4.786060333251953
2020-03-02
1.8423972129821777
1.8443925380706787
2020-03-03
1.6728532314300537
1.6758127212524414
2020-03-04
1.6278223991394043
1.6297895908355713
2020-03-05


0.9356827735900879
0.9366803169250488
2020-04-09
0.8508222103118896
0.8518192768096924
2020-04-10
1.2653710842132568
1.2673661708831787
2020-04-11
6.122638463973999
6.123635292053223
2: 740
740
3: 740
10.147961378097534
2020-02-15
3.5480806827545166
3.5510666370391846
2020-02-16
4.496382713317871
4.499366521835327
2020-02-17
1.374307632446289
1.3773000240325928
2020-02-18
1.719719409942627
1.722712755203247
2020-02-19
1.564314365386963
1.5722935199737549
2020-02-20
1.5195109844207764
1.5225028991699219
2020-02-21
1.9074714183807373
1.9104630947113037
2020-02-22
3.5500051975250244
3.552004814147949
2020-02-23
4.590718507766724
4.593710899353027
2020-02-24
1.7875456809997559
1.7905375957489014
2020-02-25
1.8046340942382812
1.8066656589508057
2020-02-26
1.6928153038024902
1.6948473453521729
2020-02-27
1.6640291213989258
1.6670501232147217
2020-02-28
1.8753929138183594
1.8784255981445312
2020-02-29
4.542973279953003
4.54596734046936
2020-03-01
5.207695484161377
5.209691762924194
2020-03-02

,FIPS,Date,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,residential_percent_change_from_baseline,retail_and_recreation_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline
0,1001,2020-02-15,7.000000,5.522819,-1.527507,5.000000,NaN,-4.0
1,1001,2020-02-16,1.000000,-23.000000,1.000000,0.000000,NaN,-4.0
2,1001,2020-02-17,0.000000,3.275069,5.000000,8.000000,NaN,-27.0
3,1001,2020-02-18,0.000000,-4.000000,0.000000,-2.000000,NaN,2.0
4,1001,2020-02-19,0.000000,5.016194,0.000000,-2.000000,NaN,2.0
5,1001,2020-02-20,-3.000000,-23.637534,2.000000,-8.000000,NaN,1.0
6,1001,2020-02-21,-2.000000,22.702300,0.000000,-3.000000,NaN,3.0
7,1001,2020-02-22,1.000000,32.506121,-2.000000,1.000000,NaN,8.0
8,1001,2020-02-23,3.000000,3.165041,-0.472493,0.000000,NaN,5.0
9,1001,2020-02-24,-5.000000,-26.055014,1.000000,-2.000000,NaN,2.0


In [179]:
Google = fillgoogle(cols[4], Google)
Google

1: 95514
16.16390633583069
2020-02-15
15.623178720474243
15.627169370651245
2020-02-16
14.43853235244751
14.447523832321167
2020-02-17
14.513951778411865
14.52292251586914
2020-02-18
14.84580683708191
14.84878134727478
2020-02-19
15.609782218933105
15.612774133682251
2020-02-20
16.9144287109375
16.917420148849487
2020-02-21
17.093823671340942
17.096787452697754
2020-02-22
15.0032217502594
15.007248401641846
2020-02-23
20.46149730682373
20.46448040008545
2020-02-24
14.098173141479492
14.10110878944397
2020-02-25
13.929596185684204
13.933586597442627
2020-02-26
16.345906734466553
16.350903272628784
2020-02-27
15.037339448928833
15.040333032608032
2020-02-28
16.382135152816772
16.388224363327026
2020-02-29
16.254836559295654
16.259855031967163
2020-03-01
14.356748580932617
14.358778953552246
2020-03-02
16.3968722820282
16.39886736869812
2020-03-03
14.473055839538574
14.47604513168335
2020-03-04
14.637672185897827
14.639628648757935
2020-03-05
14.194075345993042
14.195109128952026
2020-03-

7.955682992935181
7.957704544067383
2020-04-11
11.302818298339844
11.307803392410278
2: 1534
1: 1534
15.502877473831177
2020-02-15
5.564401626586914
5.56935715675354
2020-02-16
8.60262155532837
8.617945671081543
2020-02-17
5.111520051956177
5.118494272232056
2020-02-18
4.089213132858276
4.0951988697052
2020-02-19
2.3640732765197754
2.36806583404541
2020-02-20
2.1040382385253906
2.1080520153045654
2020-02-21
2.8654868602752686
2.868480920791626
2020-02-22
4.7707200050354
4.773715019226074
2020-02-23
5.547733306884766
5.551758289337158
2020-02-24
2.2840490341186523
2.2870452404022217
2020-02-25
2.2813236713409424
2.284315347671509
2020-02-26
2.4172329902648926
2.4212210178375244
2020-02-27
2.7488622665405273
2.7518551349639893
2020-02-28
3.5137739181518555
3.51676869392395
2020-02-29
5.770003080368042
5.772996187210083
2020-03-01
15.857019901275635
15.866023302078247
2020-03-02
3.2243099212646484
3.2332870960235596
2020-03-03
2.427821397781372
2.432807683944702
2020-03-04
2.5315759181976

1.3607304096221924
1.365717887878418
2020-04-10
2.114499568939209
2.1154961585998535
2020-04-11
9.865906000137329
9.867900133132935
2: 773
1: 773
15.842867374420166
2020-02-15
6.267768383026123
6.281734943389893
2020-02-16
12.433230876922607
12.437220096588135
2020-02-17
2.696474313735962
2.7065603733062744
2020-02-18
3.3275630474090576
3.332557439804077
2020-02-19
2.1548893451690674
2.158911943435669
2020-02-20
1.8452529907226562
1.8492414951324463
2020-02-21
2.419752836227417
2.4236843585968018
2020-02-22
4.378513336181641
4.382503032684326
2020-02-23
7.916354417800903
7.936310768127441
2020-02-24
3.9406332969665527
3.9559755325317383
2020-02-25
2.7389907836914062
2.74198317527771
2020-02-26
4.2043070793151855
4.215957403182983
2020-02-27
3.2226197719573975
3.237177610397339
2020-02-28
2.936833143234253
2.9408209323883057
2020-02-29
6.937117576599121
6.942102909088135
2020-03-01
7.06726336479187
7.0692596435546875
2020-03-02
2.2338874340057373
2.236881971359253
2020-03-03
2.645830869

1.4187822341918945
1.4198155403137207
2020-04-08
2.2034223079681396
2.2153878211975098
2020-04-09
2.0331711769104004
2.035163640975952
2020-04-10
5.217296838760376
5.225291728973389
2020-04-11
11.431187868118286
11.435171604156494
2: 754
1: 754
15.531313180923462
2020-02-15
8.554917097091675
8.583123445510864
2020-02-16
11.894552230834961
11.906533002853394
2020-02-17
2.0532000064849854
2.0631730556488037
2020-02-18
3.7563159465789795
3.764305353164673
2020-02-19
4.819496154785156
4.8294689655303955
2020-02-20
2.72965407371521
2.746610164642334
2020-02-21
4.941148519515991
4.945138216018677
2020-02-22
5.871658086776733
5.8756585121154785
2020-02-23
6.037798881530762
6.040759563446045
2020-02-24
2.1056981086730957
2.109678268432617
2020-02-25
2.0692949295043945
2.0752789974212646
2020-02-26
2.010171413421631
2.0161564350128174
2020-02-27
1.9211125373840332
1.9241423606872559
2020-02-28
2.20444393157959
2.2074482440948486
2020-02-29
4.724148511886597
4.726106882095337
2020-03-01
7.310435

1.9551353454589844
1.9561007022857666
2020-04-07
1.4876132011413574
1.49363374710083
2020-04-08
1.4738035202026367
1.4757957458496094
2020-04-09
1.1340429782867432
1.1360507011413574
2020-04-10
1.852757215499878
1.8547790050506592
2020-04-11
8.942233085632324
8.944193124771118
2: 748
1: 748
15.523340225219727
2020-02-15
5.864068508148193
5.873044490814209
2020-02-16
9.767919540405273
9.773905992507935
2020-02-17
4.2590861320495605
4.264073133468628
2020-02-18
4.121628761291504
4.125578880310059
2020-02-19
2.7372865676879883
2.7432844638824463
2020-02-20
1.7147135734558105
1.717710256576538
2020-02-21
2.834109306335449
2.8381402492523193
2020-02-22
5.136155366897583
5.139187574386597
2020-02-23
5.422855854034424
5.425851106643677
2020-02-24
2.140787124633789
2.145773410797119
2020-02-25
2.0715413093566895
2.0745294094085693
2020-02-26
2.146117925643921
2.1501212120056152
2020-02-27
2.381108283996582
2.3850982189178467
2020-02-28
3.3917760848999023
3.4033217430114746
2020-02-29
6.6801767

3.7667131423950195
3.7687246799468994
2020-04-05
4.731386423110962
4.733647346496582
2020-04-06
1.4020798206329346
1.4041125774383545
2020-04-07
1.1299397945404053
1.1319708824157715
2020-04-08
0.9577641487121582
0.9587576389312744
2020-04-09
0.8427228927612305
0.84375
2020-04-10
1.2285716533660889
1.2295680046081543
2020-04-11
6.98157525062561
6.982571601867676
2: 744
1: 744
10.624144554138184
2020-02-15
3.6373441219329834
3.6413426399230957
2020-02-16
4.671915292739868
4.675896167755127
2020-02-17
1.4064254760742188
1.410447359085083
2020-02-18
1.7635650634765625
1.7685515880584717
2020-02-19
1.763925552368164
1.768916130065918
2020-02-20
1.5696477890014648
1.5736017227172852
2020-02-21
1.931107759475708
1.9351236820220947
2020-02-22
3.587468385696411
3.59049654006958
2020-02-23
4.601729869842529
4.60476279258728
2020-02-24
1.7916193008422852
1.794611930847168
2020-02-25
1.7725379467010498
1.7765207290649414
2020-02-26
1.7074120044708252
1.710404634475708
2020-02-27
1.923248767852783

0.8450767993927002
0.8460767269134521
2020-04-03
1.133547067642212
1.1355321407318115
2020-04-04
3.2944061756134033
3.2964279651641846
2020-04-05
4.179321527481079
4.1803483963012695
2020-04-06
1.228407621383667
1.2294046878814697
2020-04-07
0.9837443828582764
0.9847416877746582
2020-04-08
0.9540002346038818
0.9559578895568848
2020-04-09
0.8425881862640381
0.8435888290405273
2020-04-10
1.2100303173065186
1.2110638618469238
2020-04-11
6.07292628288269
6.073962688446045
2: 741
741
3: 741
10.682212352752686
2020-02-15
3.6663525104522705
3.6693453788757324
2020-02-16
4.514209985733032
4.51819920539856
2020-02-17
1.376371145248413
1.3813955783843994
2020-02-18
1.7129716873168945
1.7169618606567383
2020-02-19
1.5960698127746582
1.6000487804412842
2020-02-20
1.4757909774780273
1.4787876605987549
2020-02-21
1.915919542312622
1.9189112186431885
2020-02-22
3.490985631942749
3.494011878967285
2020-02-23
4.583200454711914
4.587204217910767
2020-02-24
1.7818803787231445
1.7849085330963135
2020-02-2

,FIPS,Date,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,residential_percent_change_from_baseline,retail_and_recreation_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline
0,1001,2020-02-15,7.000000,5.522819,-1.527507,5.000000,6.364064,-4.0
1,1001,2020-02-16,1.000000,-23.000000,1.000000,0.000000,-5.888442,-4.0
2,1001,2020-02-17,0.000000,3.275069,5.000000,8.000000,-2.629827,-27.0
3,1001,2020-02-18,0.000000,-4.000000,0.000000,-2.000000,-7.607523,2.0
4,1001,2020-02-19,0.000000,5.016194,0.000000,-2.000000,-1.303654,2.0
5,1001,2020-02-20,-3.000000,-23.637534,2.000000,-8.000000,-1.296076,1.0
6,1001,2020-02-21,-2.000000,22.702300,0.000000,-3.000000,2.463812,3.0
7,1001,2020-02-22,1.000000,32.506121,-2.000000,1.000000,17.317125,8.0
8,1001,2020-02-23,3.000000,3.165041,-0.472493,0.000000,6.859852,5.0
9,1001,2020-02-24,-5.000000,-26.055014,1.000000,-2.000000,-7.894574,2.0


In [180]:
Google = fillgoogle(cols[5], Google)
Google

1: 9029
12.877569675445557
2020-02-15
10.253200054168701
10.257189989089966
2020-02-16
8.6766996383667
8.679713726043701
2020-02-17
1.7409164905548096
1.7419123649597168
2020-02-18
1.9901447296142578
1.9931371212005615
2020-02-19
1.8350696563720703
1.8370647430419922
2020-02-20
1.9428446292877197
1.9448399543762207
2020-02-21
2.530911922454834
2.5339245796203613
2020-02-22
7.902541160583496
7.904534578323364
2020-02-23
8.212558507919312
8.214582920074463
2020-02-24
2.140043020248413
2.1430351734161377
2020-02-25
2.103356122970581
2.1053507328033447
2020-02-26
2.017761468887329
2.0197560787200928
2020-02-27
2.0986268520355225
2.100621223449707
2020-02-28
2.6634817123413086
2.666476249694824
2020-02-29
7.8300182819366455
7.833041667938232
2020-03-01
8.441814661026001
8.443809509277344
2020-03-02
2.442260503768921
2.444270372390747
2020-03-03
1.8610782623291016
1.8630759716033936
2020-03-04
1.918548345565796
1.91957688331604
2020-03-05
1.961611270904541
1.9645764827728271
2020-03-06
2.729

1.4573984146118164
1.4584057331085205
2020-04-11
6.835809946060181
6.836810827255249
2: 741
1: 741
11.815746784210205
2020-02-15
4.137953519821167
4.140934228897095
2020-02-16
5.175057888031006
5.178048610687256
2020-02-17
1.5481364727020264
1.5511295795440674
2020-02-18
2.111201047897339
2.114164352416992
2020-02-19
1.9001781940460205
1.9021360874176025
2020-02-20
1.5766828060150146
1.5796890258789062
2020-02-21
2.003730297088623
2.006721019744873
2020-02-22
3.7571792602539062
3.7601702213287354
2020-02-23
4.890087604522705
4.893045902252197
2020-02-24
1.9589297771453857
1.9618933200836182
2020-02-25
1.9463226795196533
1.9492785930633545
2020-02-26
1.8167691230773926
1.8187639713287354
2020-02-27
1.8880479335784912
1.891040563583374
2020-02-28
2.193950653076172
2.195950508117676
2020-02-29
4.353840589523315
4.356834888458252
2020-03-01
5.975249290466309
5.97923731803894
2020-03-02
2.280644416809082
2.28363037109375
2020-03-03
1.9997918605804443
2.001788377761841
2020-03-04
1.784477949

,FIPS,Date,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,residential_percent_change_from_baseline,retail_and_recreation_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline
0,1001,2020-02-15,7.000000,5.522819,-1.527507,5.000000,6.364064,-4.000000
1,1001,2020-02-16,1.000000,-23.000000,1.000000,0.000000,-5.888442,-4.000000
2,1001,2020-02-17,0.000000,3.275069,5.000000,8.000000,-2.629827,-27.000000
3,1001,2020-02-18,0.000000,-4.000000,0.000000,-2.000000,-7.607523,2.000000
4,1001,2020-02-19,0.000000,5.016194,0.000000,-2.000000,-1.303654,2.000000
5,1001,2020-02-20,-3.000000,-23.637534,2.000000,-8.000000,-1.296076,1.000000
6,1001,2020-02-21,-2.000000,22.702300,0.000000,-3.000000,2.463812,3.000000
7,1001,2020-02-22,1.000000,32.506121,-2.000000,1.000000,17.317125,8.000000
8,1001,2020-02-23,3.000000,3.165041,-0.472493,0.000000,6.859852,5.000000
9,1001,2020-02-24,-5.000000,-26.055014,1.000000,-2.000000,-7.894574,2.000000


In [184]:
Google = Google.fillna(0)
Google.to_csv('google_new_filled.csv')